# Traveling Salesman Problem Algorithm using Genetic Algorithms

This notebook implements a genetic algorithm, that tries to "solve" the traveling salesman problem.

In [127]:
import random
import math
from IPython.core.debugger import set_trace
from IPython.display import clear_output

In [128]:
cities = []
numCities = 10

# The total size of the area, in which we will generate the cities
areaX = 20
areaY = 20

bestEver = []
recordDistance = 1e7
currentBest = []

population = []
numPopulation = 400
fitness = []

masterOrder = [x for x in range(numCities)]
# generate a random cities with position tuples of (x,y)
for i in range(numCities):
    newCity = (random.randint(1,areaX), random.randint(1,areaX))
    
    while newCity in cities:
        newCity = (random.randint(1,areaX), random.randint(1,areaY))
        
    cities.append(newCity)
    

In [129]:
# generate a population for the genetic algorithm
for i in range(numPopulation):
    order = random.sample(masterOrder, k=len(masterOrder))
    population.append(order)

In [130]:
# calculates the distance of a give order
def calcDistance(order):
    dist = 0
    for i in range(len(order)-1):
        #set_trace()
        dist += math.sqrt( (cities[order[i+1]][0] - cities[order[i]][0])**2 + (cities[order[i+1]][1] - cities[order[i]][1])**2 )
    return dist

In [131]:
# calculates the fitness values for the generated population
def calculateFitness():
    currentRecord = 1e7
    global recordDistance, bestEver
    for i in range(numPopulation):
        # calculate the distance of the orders in the population
        distance = calcDistance(population[i])
        
        # if the ditance is better than what we have ever seen before, save it
        if distance < recordDistance:
            recordDistance = distance
            bestEver = population[i]
        
        # if the ditance is better than what we have seen before in the population, save it
        if distance < currentRecord:
            currentRecord = distance
            currentBest = population[i]
            
        fitness.append( 1 / (pow(distance, 8) + 1))
        

In [132]:
# normalize the fitness
def normalizeFitness():
    theSUM = 0
    for i in range(len(fitness)):
        theSUM += fitness[i]
    for i in range(len(fitness)):
        fitness[i] = fitness[i] / theSUM

In [133]:
# pickes an element of a list by it's given probability
def pickOne(ls,prob):
    index = 0
    r = random.uniform(0,1)
    
    while r > 0:
        r = r - prob[index]
        index += 1
    
    index -= 1
    
    return ls[index]

In [134]:
# randomly mutates out order
def mutate(order, mutationRate):
    for i in range(numCities):
        if random.uniform(0,1) < mutationRate:
            index1 = math.floor(random.randint(0, len(order)-1))
            index2 = math.floor(random.randint(0, len(order)-1))
            #set_trace()
            order[index1], order[index2] = order[index2], order[index1]
            

In [135]:
def crossOver(order1, order2):
    start = math.floor(random.randint(0,len(order1)))
    end = math.floor(random.randint(start,len(order1)))
    neworder = order1[start:end]
    #set_trace()
    for i in range(len(order2)):
        city = order2[i]
        if city not in neworder:
            neworder.append(city)
    return neworder

In [136]:
def nextGeneration():
    newPopulation = []
    global population
    for i in range(numPopulation):
        order1 = pickOne(population, fitness)
        order2 = pickOne(population, fitness)
        order = crossOver(order1, order2)
        mutate(order,0.05)
        newPopulation.append(order)
    population = newPopulation

In [137]:
%%time
print("Searching good path...")

for i in range(1000):
    calculateFitness()
    normalizeFitness()
    nextGeneration()
    print ("[" + str(i) + "/1000" +"] " + "Current best order: " + str(bestEver) + " with a distance of " + str(recordDistance), end="\r")
    
clear_output()
print("Best path found: " + str(bestEver) + " with a length of " + str(recordDistance))

Best path found: [7, 1, 3, 2, 6, 5, 8, 9, 4, 0] with a length of 40.645455487423945
CPU times: user 29.5 s, sys: 48 ms, total: 29.6 s
Wall time: 29.5 s
